In [72]:
# 59a373a01cdecc10846173f40e09948d62e51b96b6c2333a1464dace93654403
# 174e83de9fd1e7683780574493d40afb35a40ab10c09fbce05ece7e145f04f8b

import pefile
import re
from capstone import *
from capstone.x86 import *

pat = b'\xe8\xf4\xfe\xff\xff'

pe = pefile.PE(r"") # Put your file path

# Initialize Capstone
md = Cs(CS_ARCH_X86, CS_MODE_32)
md.skipdata = True
md.detail = True

In [74]:


insnts = []
text_section = pe.sections[0]
text_section_addr = pe.OPTIONAL_HEADER.ImageBase + text_section.VirtualAddress

# Disassemble .text section
for insnt in md.disasm(text_section.get_data(), text_section_addr):
    insnts.append(insnt)
    # print(insnt)
sd = 0
ip_list = []
for i in range(len(insnts)):
    insn = insnts[i]
    # th_inst = insnts[i+2]
    if (insn.mnemonic == "mov" and  insn.operands[0].type == X86_OP_MEM and
        insn.operands[1].type == X86_OP_IMM):
        next_insn = insnts[i+1]
        if (next_insn.mnemonic == "mov" and  next_insn.operands[0].type == X86_OP_MEM and
        next_insn.operands[1].type == X86_OP_IMM):
            thi_insn = insnts[i+2]
            if (thi_insn.mnemonic == "mov" and  thi_insn.operands[0].type == X86_OP_MEM and
        thi_insn.operands[1].type == X86_OP_IMM):
                if insn.operands[1].imm == 0:
                # print(insn)
                    first_inst = insnts[i]
                    # print(first_inst.bytes)
                    # print(first_inst.address)
                    for j in range (0,16):
                        mov_inst = insnts[i+j]
                        ip_list.append(chr(mov_inst.operands[1].imm) )

    if re.match(pat, insnts[i].bytes):
        port_inst = insnts[i-1]
        port = port_inst.operands[1].imm


print("C2:", end="")                       
for i in ip_list:
    print(i, end='')

print("\nPort: ", port )

C2: 146.70.169.164 
Port:  2227
